In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
    
class ModelConfig(object):
    embeddingSize = 200
    
    filters = 128  # 内层一维卷积核的数量，外层卷积核的数量应该等于embeddingSize
    numHeads = 8  # Attention 的头数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 生成位置嵌入
def positionEmbedding(batchSize, sequenceLen):
    embeddedPosition = []
    for batch in range(batchSize):
        x = []
        for step in range(sequenceLen):
            a = np.zeros(sequenceLen)
            a[step] = 1
            x.append(a)
        embeddedPosition.append(x)
    
    return np.array(embeddedPosition, dtype="float32")

In [7]:
# 模型构建

class Transformer(object):
    """
    Transformer Encoder 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.embeddedPosition = tf.placeholder(tf.float32, [None, config.sequenceLength, config.sequenceLength], name="embeddedPosition")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embedded = tf.nn.embedding_lookup(self.W, self.inputX)
            self.embeddedWords = tf.concat([self.embedded, self.embeddedPosition], -1)
        
            
        with tf.name_scope("transformer"):
            
            # 维度[batch_size, sequence_length, embedding_size]
            multiHeadAtt = self.multiheadAttention(queries=self.embeddedWords, keys=self.embeddedWords)
            # 维度[batch_size, sequence_length, embedding_size]
            outputs = self.feedForward(multiHeadAtt, [config.model.filters, config.model.embeddingSize + config.sequenceLength])
            outputs = tf.reshape(outputs, [-1, config.sequenceLength * (config.model.embeddingSize + config.sequenceLength)])

        outputSize = outputs.get_shape()[-1].value
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(outputs, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
            
    def layerNormalization(self, inputs, epsilon=1e-8, scope="layerNorm", reuse=None):
        # 本质上可以看作是BN层
        
        with tf.variable_scope(scope, reuse=reuse):
            inputsShape = inputs.get_shape() # [batch_size, sequence_length, embedding_size]
            paramsShape = inputsShape[-1:]
  
            # 在最后的维度上计算输入的数据的均值和方差
            mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
            beta = tf.Variable(tf.zeros(paramsShape))
            gamma = tf.Variable(tf.ones(paramsShape))
            normalized = (inputs - mean) / ((variance + epsilon) ** .5)
            outputs = gamma * normalized + beta

        return outputs
            
    def multiheadAttention(self, queries, keys, numUnits=None, numHeads=8, causality=False, scope="multiheadAttention", reuse=None):
        
        with tf.variable_scope(scope, reuse=reuse):
            if numUnits is None:  # 若是没传入值，直接去输入数据的最后一维，即embedding size.
                numUnits = queries.get_shape().as_list()[-1]

            # tf.layers.dense可以做多维tensor数据的非线性映射，在计算self-Attention时，一定要对这三个值进行非线性映射，
            # 其实这一步就是论文中Multi-Head Attention中的对分割后的数据进行权重映射的步骤，我们在这里先映射后分割，原则上是一样的。
            # Q, K, V的维度都是[batch_size, sequence_length, embedding_size]
            Q = tf.layers.dense(queries, numUnits, activation=tf.nn.relu)
            K = tf.layers.dense(keys, numUnits, activation=tf.nn.relu)
            V = tf.layers.dense(keys, numUnits, activation=tf.nn.relu)
            
            # 将数据按最后一维分割成num_heads个, 然后按照第一维拼接
            # Q, K, V 的维度都是[batch_size * numHeads, sequence_length, embedding_size/numHeads]
            Q_ = tf.concat(tf.split(Q, numHeads, axis=-1), axis=0) 
            K_ = tf.concat(tf.split(K, numHeads, axis=-1), axis=0) 
            V_ = tf.concat(tf.split(V, numHeads, axis=-1), axis=0)
            
            # 计算keys和queries之间的点积，维度[batch_size * numHeads, queries_len, key_len], 后两维是queries和keys的序列长度
            similary = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1]))
            
            # 对计算的点积进行缩放处理，除以向量长度的根号值
            scaledSimilary = similary / (K_.get_shape().as_list()[-1] ** 0.5)
            
            # 在我们输入的序列中会存在padding这个样的填充词，这种词应该对最终的结果是毫无帮助的，原则上说当padding都是输入0时，
            # 计算出来的权重应该也是0，但是在transformer中引入了位置向量，当和位置向量相加之后，其值就不为0了，因此在添加位置向量
            # 之前，我们需要将其mask为0。虽然在queries中也存在这样的填充词，但原则上模型的结果之和输入有关，而且在self-Attention中
            # queryies = keys，因此只要一方为0，计算出的权重就为0。
            # 具体关于key mask的介绍可以看看这里： https://github.com/Kyubyong/transformer/issues/3
            
            # 将每一时序上的向量中的值相加取平均值
            keyMasks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1)))  # 维度[batch_size, time_step]
            
            # 利用tf，tile进行张量扩张， 维度[batch_size * numHeads, keys_len] keys_len = keys 的序列长度
            keyMasks = tf.tile(keyMasks, [numHeads, 1]) 
            
            # 增加一个维度，并进行扩张，得到维度[batch_size * numHeads, queries_len, keys_len]
            keyMasks = tf.tile(tf.expand_dims(keyMasks, 1), [1, tf.shape(queries)[1], 1])
            
            # tf.ones_like生成元素全为1，维度和scaledSimilary相同的tensor, 然后得到负无穷大的值
            paddings = tf.ones_like(scaledSimilary) * (-2 ** (32 + 1))
            
            # tf.where(condition, x, y),condition中的元素为bool值，其中对应的True用x中的元素替换，对应的False用y中的元素替换
            # 因此condition,x,y的维度是一样的。下面就是keyMasks中的值为0就用paddings中的值替换
            maskedSimilary = tf.where(tf.equal(keyMasks, 0), paddings, scaledSimilary) # 维度[batch_size * numHeads, queries_len, key_len]
            
            # 在计算当前的词时，只考虑上文，不考虑下文，出现在Transformer Decoder中。在文本分类时，可以只用Transformer Encoder。
            # Decoder是生成模型，主要用在语言生成中
            if causality:
                diagVals = tf.ones_like(maskedSimilary[0, :, :])  # [queries_len, keys_len]
                tril = tf.contrib.linalg.LinearOperatorTriL(diagVals).to_dense()  # [queries_len, keys_len]
                masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(maskedSimilary)[0], 1, 1])  # [batch_size * numHeads, queries_len, keys_len]

                paddings = tf.ones_like(masks) * (-2 ** (32 + 1))
                maskedSimilary = tf.where(tf.equal(masks, 0), paddings, maskedSimilary)  # [batch_size * numHeads, queries_len, keys_len]

            # 通过softmax计算权重系数，维度 [batch_size * numHeads, queries_len, keys_len]
            weights = tf.nn.softmax(maskedSimilary)

            # 加权和得到输出值, 维度[batch_size * numHeads, sequence_length, embedding_size/numHeads]
            outputs = tf.matmul(weights, V_)
            
            # 将多头Attention计算的得到的输出重组成最初的维度[batch_size, sequence_length, embedding_size]
            outputs = tf.concat(tf.split(outputs, numHeads, axis=0), axis=2)

            # 建立残差连接
            outputs += queries
            # normalization 层
            outputs = self.layerNormalization(outputs)
            return outputs

    def feedForward(self, inputs, filters, scope="multiheadAttention", reuse=None):
        # 在这里的前向传播采用卷积神经网络
        with tf.variable_scope(scope, reuse=reuse):
            # 内层
            params = {"inputs": inputs, "filters": filters[0], "kernel_size": 1,
                      "activation": tf.nn.relu, "use_bias": True}
            outputs = tf.layers.conv1d(**params)

            # 外层
            params = {"inputs": outputs, "filters": filters[1], "kernel_size": 1,
                      "activation": None, "use_bias": True}
            
            # 这里用到了一维卷积，实际上卷积核尺寸还是二维的，只是只需要指定高度，宽度和embedding size的尺寸一致
            # 维度[batch_size, sequence_length, embedding_size]
            outputs = tf.layers.conv1d(**params)
            
            # 残差连接
            outputs += inputs

            # 归一化处理
            outputs = self.layerNormalization(outputs)

        return outputs

In [8]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [9]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

embeddedPosition = positionEmbedding(config.batchSize, config.sequenceLength)

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        transformer = Transformer(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(transformer.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", transformer.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Transformer/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              transformer.inputX: batchX,
              transformer.inputY: batchY,
              transformer.dropoutKeepProb: config.model.dropoutKeepProb,
              transformer.embeddedPosition: embeddedPosition
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, transformer.loss, transformer.predictions, transformer.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              transformer.inputX: batchX,
              transformer.inputY: batchY,
              transformer.dropoutKeepProb: 1.0,
              transformer.embeddedPosition: embeddedPosition
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, transformer.loss, transformer.predictions, transformer.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Transformer/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(transformer.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(transformer.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(transformer.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name multiheadAttention/dense/kernel:0/grad/hist is illegal; using multiheadAttention/dense/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name multiheadAttention/dense/kernel:0/grad/sparsity is illegal; using multiheadAttention/dense/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name multiheadAttention/dense/bias:0/grad/hist is illegal; using multiheadAttention/dense/bias_0/grad/hist instead.
INFO:tensorflow:Summary name multiheadAttention/dense/bias:0/grad/sparsity is illegal; using multiheadAttention/dense/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name multiheadAttention/dense_1/kernel:0/grad/hist is illegal; using multiheadAttention/dense_1/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name multiheadAttention/dens

2019-01-07T15:28:56.481019, step: 26, loss: 1.6133065223693848, acc: 0.4688, auc: 0.5696, precision: 0.4649, recall: 0.8833
2019-01-07T15:28:56.691030, step: 27, loss: 0.9087508916854858, acc: 0.625, auc: 0.6146, precision: 0.6477, recall: 0.7703
2019-01-07T15:28:56.894477, step: 28, loss: 0.8405101299285889, acc: 0.5781, auc: 0.6136, precision: 0.5641, recall: 0.3729
2019-01-07T15:28:57.105792, step: 29, loss: 1.3453201055526733, acc: 0.5312, auc: 0.621, precision: 0.7143, recall: 0.0794
2019-01-07T15:28:57.308600, step: 30, loss: 1.050517201423645, acc: 0.5703, auc: 0.5919, precision: 0.619, recall: 0.2167
2019-01-07T15:28:57.523163, step: 31, loss: 0.7605966925621033, acc: 0.6328, auc: 0.7153, precision: 0.5616, recall: 0.7321
2019-01-07T15:28:57.736532, step: 32, loss: 1.038586139678955, acc: 0.5625, auc: 0.6392, precision: 0.5125, recall: 0.7069
2019-01-07T15:28:57.945155, step: 33, loss: 0.8473740816116333, acc: 0.5859, auc: 0.6155, precision: 0.6604, recall: 0.5
2019-01-07T15:28

2019-01-07T15:29:10.658423, step: 93, loss: 0.4956212043762207, acc: 0.8359, auc: 0.9229, precision: 0.7571, recall: 0.9298
2019-01-07T15:29:10.861933, step: 94, loss: 0.5124728083610535, acc: 0.7578, auc: 0.9034, precision: 0.875, recall: 0.6269
2019-01-07T15:29:11.062429, step: 95, loss: 0.6336507797241211, acc: 0.7422, auc: 0.9277, precision: 0.9697, recall: 0.5
2019-01-07T15:29:11.266777, step: 96, loss: 0.7290962338447571, acc: 0.75, auc: 0.8962, precision: 0.9429, recall: 0.5238
2019-01-07T15:29:11.479876, step: 97, loss: 0.528708279132843, acc: 0.7891, auc: 0.8879, precision: 0.8033, recall: 0.7656
2019-01-07T15:29:11.681565, step: 98, loss: 0.716462254524231, acc: 0.8047, auc: 0.9116, precision: 0.7419, recall: 0.9857
2019-01-07T15:29:11.895444, step: 99, loss: 0.6416012048721313, acc: 0.8203, auc: 0.911, precision: 0.7753, recall: 0.9583
2019-01-07T15:29:12.097399, step: 100, loss: 0.424837201833725, acc: 0.8359, auc: 0.9182, precision: 0.8243, recall: 0.8841

Evaluation:
2019

2019-01-07T15:29:32.231983, step: 158, loss: 0.27248311042785645, acc: 0.9062, auc: 0.9682, precision: 0.875, recall: 0.9545
2019-01-07T15:29:32.438146, step: 159, loss: 0.37943917512893677, acc: 0.8672, auc: 0.9543, precision: 0.9423, recall: 0.7778
2019-01-07T15:29:32.644117, step: 160, loss: 0.35967573523521423, acc: 0.75, auc: 0.9887, precision: 1.0, recall: 0.5294
2019-01-07T15:29:32.859794, step: 161, loss: 0.1757589876651764, acc: 0.9297, auc: 0.9801, precision: 0.9474, recall: 0.9
2019-01-07T15:29:33.057696, step: 162, loss: 0.24997590482234955, acc: 0.9297, auc: 0.9621, precision: 0.9315, recall: 0.9444
2019-01-07T15:29:33.270960, step: 163, loss: 0.34177762269973755, acc: 0.9062, auc: 0.9811, precision: 0.8519, recall: 1.0
2019-01-07T15:29:33.485518, step: 164, loss: 0.1952819526195526, acc: 0.9375, auc: 0.9765, precision: 0.9277, recall: 0.9747
2019-01-07T15:29:33.696434, step: 165, loss: 0.23345084488391876, acc: 0.9141, auc: 0.9694, precision: 0.9079, recall: 0.9452
2019-0

2019-01-07T15:29:53.950559, step: 223, loss: 0.15075638890266418, acc: 0.9297, auc: 0.9887, precision: 0.9636, recall: 0.8833
2019-01-07T15:29:54.167966, step: 224, loss: 0.18247927725315094, acc: 0.9062, auc: 0.9836, precision: 0.963, recall: 0.8387
2019-01-07T15:29:54.371264, step: 225, loss: 0.17290566861629486, acc: 0.9375, auc: 0.9829, precision: 1.0, recall: 0.8788
2019-01-07T15:29:54.585385, step: 226, loss: 0.27522599697113037, acc: 0.9141, auc: 0.9685, precision: 0.8857, recall: 0.9538
2019-01-07T15:29:54.793792, step: 227, loss: 0.1898288130760193, acc: 0.9219, auc: 0.985, precision: 0.9153, recall: 0.9153
2019-01-07T15:29:55.005103, step: 228, loss: 0.1659417450428009, acc: 0.9297, auc: 0.9841, precision: 0.9322, recall: 0.9167
2019-01-07T15:29:55.215854, step: 229, loss: 0.2313946783542633, acc: 0.8906, auc: 0.9713, precision: 0.9091, recall: 0.8824
2019-01-07T15:29:55.431195, step: 230, loss: 0.3610122799873352, acc: 0.8906, auc: 0.9462, precision: 0.9592, recall: 0.7966
2

...

2019-01-07T15:34:52.293520, step: 1260, loss: 0.0003939501184504479, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:34:52.502671, step: 1261, loss: 0.0005556398537009954, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:34:52.703717, step: 1262, loss: 0.0008131119539029896, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:34:52.909042, step: 1263, loss: 0.0004853864083997905, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:34:53.138378, step: 1264, loss: 0.000532378675416112, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:34:53.351142, step: 1265, loss: 0.00046312014455907047, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:34:53.564868, step: 1266, loss: 0.0006026573828421533, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:34:53.775414, step: 1267, loss: 0.0003769598843064159, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:34:53.979392, step: 1268, loss: 0.0006300652166828

2019-01-07T15:35:14.898818, step: 1330, loss: 0.0006650331779383123, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:15.108875, step: 1331, loss: 0.00031096814200282097, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:15.328672, step: 1332, loss: 0.00045247527305036783, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:15.527242, step: 1333, loss: 0.0006128564709797502, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:15.736636, step: 1334, loss: 0.0006492679822258651, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:15.939770, step: 1335, loss: 0.0005846734857186675, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:16.154174, step: 1336, loss: 0.00075154984369874, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:16.366008, step: 1337, loss: 0.0005607332568615675, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:16.589355, step: 1338, loss: 0.0008520944975316

2019-01-07T15:35:29.520610, step: 1400, loss: 0.0004427750827744603, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0

Evaluation:
2019-01-07T15:35:37.549059, step: 1400, loss: 0.5615845998892417, acc: 0.8695948717948718, auc: 0.9452871794871794, precision: 0.8832846153846153, recall: 0.8551641025641025
2019-01-07T15:35:37.746724, step: 1401, loss: 0.000510662270244211, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:37.951839, step: 1402, loss: 0.0005118739791214466, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:38.160862, step: 1403, loss: 0.0008077961392700672, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:38.370180, step: 1404, loss: 0.0005375779001042247, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
start training model
2019-01-07T15:35:38.608247, step: 1405, loss: 0.00048589176731184125, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:38.817592, step: 1406, loss: 0.0005836407653987408, acc: 1.0, auc: 1.0, p

2019-01-07T15:35:51.895038, step: 1469, loss: 0.0005925646401010454, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:52.117311, step: 1470, loss: 0.00032878475030884147, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:52.329788, step: 1471, loss: 0.00040350761264562607, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:52.531058, step: 1472, loss: 0.0003993118880316615, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:52.741926, step: 1473, loss: 0.00026039162185043097, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:52.944243, step: 1474, loss: 0.0004996603238396347, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:53.157039, step: 1475, loss: 0.0005835354095324874, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:53.371230, step: 1476, loss: 0.0004425386432558298, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:35:53.585483, step: 1477, loss: 0.0005303866346

2019-01-07T15:36:14.349172, step: 1538, loss: 0.00025316531537100673, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:36:14.561757, step: 1539, loss: 0.0002845361887011677, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:36:14.783980, step: 1540, loss: 0.0003060493036173284, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:36:14.991985, step: 1541, loss: 0.0006339049432426691, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:36:15.176406, step: 1542, loss: 0.0004735615220852196, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:36:15.373553, step: 1543, loss: 0.0004441399360075593, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:36:15.577360, step: 1544, loss: 0.00024531918461434543, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:36:15.798863, step: 1545, loss: 0.00047307321801781654, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-01-07T15:36:15.998400, step: 1546, loss: 0.0004033737932